In [136]:
import pandas as pd
import tensorflow as tf
import sklearn

In [137]:
from tensorflow.keras.layers import Dense, Dropout, Activation, Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [138]:
base = pd.read_csv("vgsales.csv")
base

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [139]:
base = base.drop('Other_Sales', axis = 1)
base = base.drop('Global_Sales', axis = 1)

In [140]:
base.shape

(16598, 9)

In [141]:
base.isnull().sum()

Rank           0
Name           0
Platform       0
Year         271
Genre          0
Publisher     58
NA_Sales       0
EU_Sales       0
JP_Sales       0
dtype: int64

In [142]:
base = base.dropna(axis=0)

In [143]:
base.isnull().sum()

Rank         0
Name         0
Platform     0
Year         0
Genre        0
Publisher    0
NA_Sales     0
EU_Sales     0
JP_Sales     0
dtype: int64

In [144]:
base['Name'].value_counts()

Name
Need for Speed: Most Wanted    12
FIFA 14                         9
Ratatouille                     9
LEGO Marvel Super Heroes        9
Cars                            8
                               ..
PGA Tour 96                     1
Game & Wario                    1
Angry Birds                     1
Shadow Hearts: Covenant         1
Know How 2                      1
Name: count, Length: 11325, dtype: int64

In [145]:
base = base.drop('Name', axis = 1)

In [146]:
base.shape

(16291, 8)

In [147]:
base.columns

Index(['Rank', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales'],
      dtype='object')

In [148]:
X = base.iloc[: , [1, 2, 3, 4]].values
X

array([['Wii', 2006.0, 'Sports', 'Nintendo'],
       ['NES', 1985.0, 'Platform', 'Nintendo'],
       ['Wii', 2008.0, 'Racing', 'Nintendo'],
       ...,
       ['PS2', 2008.0, 'Racing', 'Activision'],
       ['DS', 2010.0, 'Puzzle', '7G//AMES'],
       ['GBA', 2003.0, 'Platform', 'Wanadoo']], dtype=object)

In [149]:
y_na = base.iloc[:, 5].values
y_eu = base.iloc[:, 6].values
y_jp = base.iloc[:, 7].values

In [150]:
y_na

array([4.149e+01, 2.908e+01, 1.585e+01, ..., 0.000e+00, 0.000e+00,
       1.000e-02])

In [151]:
y_eu

array([2.902e+01, 3.580e+00, 1.288e+01, ..., 0.000e+00, 1.000e-02,
       0.000e+00])

In [152]:
y_jp

array([3.77, 6.81, 3.79, ..., 0.  , 0.  , 0.  ])

In [153]:
base['Platform'].value_counts()

Platform
DS      2131
PS2     2127
PS3     1304
Wii     1290
X360    1234
PSP     1197
PS      1189
PC       938
XB       803
GBA      786
GC       542
3DS      499
PSV      410
PS4      336
N64      316
SNES     239
XOne     213
SAT      173
WiiU     143
2600     116
NES       98
GB        97
DC        52
GEN       27
NG        12
SCD        6
WS         6
3DO        3
TG16       2
GG         1
PCFX       1
Name: count, dtype: int64

In [154]:
onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0, 2, 3])], remainder='passthrough')
X = onehotencoder.fit_transform(X).toarray()

In [156]:
X.shape

(16291, 620)

In [157]:
input_layer = Input(shape=(620,))
hidden_layer1 = Dense(units = 313, activation = 'relu')(input_layer)
hidden_layer2 = Dense(units = 313, activation = 'relu')(hidden_layer1)
output_layer1 = Dense(units = 1, activation = 'linear')(hidden_layer2)
output_layer2 = Dense(units = 1, activation = 'linear')(hidden_layer2)
output_layer3 = Dense(units = 1, activation = 'linear')(hidden_layer2)

In [158]:
regressor = Model(inputs = input_layer, outputs = [output_layer1, output_layer2, output_layer3])

In [159]:
regressor.compile(optimizer = 'adam', loss = 'mse')

In [160]:
regressor.fit(X, [y_na, y_eu, y_jp], epochs = 500, batch_size = 100)

Epoch 1/500
163/163 ━━━━━━━━━━━━━━━━━━━━ 13s 12ms/step - dense_2_loss: 179.0430 - dense_3_loss: 1332.0630 - dense_4_loss: 310.4202 - loss: 1821.5298
Epoch 2/500
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - dense_2_loss: 0.4954 - dense_3_loss: 0.2350 - dense_4_loss: 0.1079 - loss: 0.8383
Epoch 3/500
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - dense_2_loss: 0.8798 - dense_3_loss: 0.2674 - dense_4_loss: 0.2022 - loss: 1.3494
Epoch 4/500
163/163 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - dense_2_loss: 1.5554 - dense_3_loss: 0.6987 - dense_4_loss: 0.5305 - loss: 2.7828
Epoch 5/500
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - dense_2_loss: 11.1081 - dense_3_loss: 6.5583 - dense_4_loss: 4.9923 - loss: 22.6587
Epoch 6/500
163/163 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - dense_2_loss: 0.6136 - dense_3_loss: 0.2297 - dense_4_loss: 0.1797 - loss: 1.0230
Epoch 7/500
163/163 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - dense_2_loss: 1.1593 - dense_3_loss: 0.5129 - dense_4_loss: 0.3196 - loss: 1.9918
Epoch 8/500
163/163 ━

In [161]:
prev_na, prev_eu, prev_jp = regressor.predict(X)

510/510 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
